## **CNN Modeling**

In [24]:
import numpy as np
import pandas as pd

# 벡터화된 데이터 불러오기
data = pd.read_json("./labeled_data.json")
data.columns = ["label", "trigram"]

In [25]:
# flastten: 3 x 50 -> 150
data['trigram'] = data['trigram'].apply(lambda x: (np.array(x).reshape(-1)))

In [26]:
# train/test 데이터 분리
from sklearn.model_selection import train_test_split

y = data.pop('label')
X = data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [27]:
X_train = np.array(X_train['trigram'].tolist())
X_test = np.array(X_test['trigram'].tolist())
y_train = y_train.values
y_test = y_test.values

In [28]:
# row x embeedding 
X_train.shape

(9779, 150)

In [29]:
# row x 1
y_train.shape

(9779,)

In [30]:
# keras input
def reshape(df, dim):
    return df.reshape(df.shape[0], dim, 1)

In [31]:
X_train = reshape(X_train, 150)
X_test = reshape(X_test, 150)
X_train.shape

(9779, 150, 1)